In [1]:
import boto3
region = boto3.session.Session().region_name
bucket = 'sagemaker-studio-8r6zfwskopd'

In [2]:
account=boto3.client('sts').get_caller_identity().get('Account')
repositoryUri="{}.dkr.ecr.{}.amazonaws.com/vgg16-tl-tf".format(account, region)
repositoryUri

'885692174401.dkr.ecr.ap-northeast-2.amazonaws.com/vgg16-tl-tf'

In [3]:
from datetime import datetime
now = datetime.now()
job_name = 'vgg16-' + now.strftime("%Y-%m-%d-%H-%M-%S")
job_name

'vgg16-2021-05-10-01-03-26'

In [4]:
import sagemaker
from sagemaker import get_execution_role
role = get_execution_role()

In [5]:
train='s3://{}/train/'.format(bucket)
test='s3://{}/test/'.format(bucket)
validation='s3://{}/validation/'.format(bucket)
outpath='s3://{}/model/'.format(bucket)
(train, test, validation, outpath) 

('s3://sagemaker-studio-8r6zfwskopd/train/',
 's3://sagemaker-studio-8r6zfwskopd/test/',
 's3://sagemaker-studio-8r6zfwskopd/validation/',
 's3://sagemaker-studio-8r6zfwskopd/model/')

In [6]:
sm = boto3.client('sagemaker')

response = sm.create_training_job(
      TrainingJobName=job_name,
      AlgorithmSpecification={
          'TrainingImage': repositoryUri,
          'TrainingInputMode': 'File'
      },
      RoleArn=role,
      InputDataConfig=[
          {
              'ChannelName': 'train',
              'DataSource': {
                  'S3DataSource': {
                      'S3DataType': 'S3Prefix',
                      'S3Uri': train,
                      'S3DataDistributionType': 'FullyReplicated',
                  },
              },
              'InputMode': 'File'
          },
                    {
              'ChannelName': 'test',
              'DataSource': {
                  'S3DataSource': {
                      'S3DataType': 'S3Prefix',
                      'S3Uri': test,
                      'S3DataDistributionType': 'FullyReplicated',
                  },
              },
              'InputMode': 'File'
          },
          {
              'ChannelName': 'validation',
              'DataSource': {
                  'S3DataSource': {
                      'S3DataType': 'S3Prefix',                      
                      'S3Uri': validation,
                      'S3DataDistributionType': 'FullyReplicated',
                  },
              },
              'InputMode': 'File'
          },
          
      ],
      OutputDataConfig={
          'S3OutputPath': outpath
      },
      ResourceConfig={
          'InstanceType': 'ml.g4dn.xlarge',
          'InstanceCount': 1,
          'VolumeSizeInGB': 10,
      },
      StoppingCondition={
        'MaxRuntimeInSeconds': 60*60*5,
      }
  )
response

{'TrainingJobArn': 'arn:aws:sagemaker:ap-northeast-2:885692174401:training-job/vgg16-2021-05-10-01-03-26',
 'ResponseMetadata': {'RequestId': 'c3a07419-a559-4b16-a6cc-49e34b37214f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c3a07419-a559-4b16-a6cc-49e34b37214f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '105',
   'date': 'Mon, 10 May 2021 01:03:37 GMT'},
  'RetryAttempts': 0}}